In [22]:
import tensorflow as tf
import numpy as np
import json
import io
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense
from tensorflow.keras.models import Sequential

In [32]:
!wget --no-check-certificate \
    https://storage.googleapis.com/learning-datasets/sarcasm.json \
    -O /tmp/sarcasm.json

--2023-07-11 19:42:54--  https://storage.googleapis.com/learning-datasets/sarcasm.json
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 108.177.98.128, 74.125.197.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5643545 (5.4M) [application/json]
Saving to: ‘/tmp/sarcasm.json’

/tmp/sarcasm.json   100%[===================>]   5.38M  --.-KB/s    in 0.03s   

2023-07-11 19:42:54 (172 MB/s) - ‘/tmp/sarcasm.json’ saved [5643545/5643545]



In [33]:
with open("/tmp/sarcasm.json", 'r') as f:
    datastore = json.load(f)

sentences = []
labels = []

for item in datastore:
    sentences.append(item['headline'])
    labels.append(item['is_sarcastic'])

In [34]:
tokenizer = Tokenizer(num_words = 10000, oov_token="<OOV>")
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index

In [35]:
sequences = tokenizer.texts_to_sequences(sentences)

In [36]:
padded = pad_sequences(sequences, padding="post", truncating="post", maxlen=max(len(lst) for lst in sequences))

In [37]:
train_sentences = np.array(padded[:8000])
train_labels = np.array(labels[:8000])
test_sentences = np.array(padded[8000:])
test_labels = np.array(labels[8000:])

In [38]:
model = Sequential()
model.add(Embedding(input_dim=10000, output_dim=16, input_length=max(len(lst) for lst in sequences)))
model.add(GlobalAveragePooling1D())
model.add(Dense(units=24, activation="relu"))
model.add(Dense(units=1, activation="sigmoid"))

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy'])

In [39]:
model.fit(train_sentences, train_labels, epochs=10, validation_data=(test_sentences, test_labels))

Epoch 1/10
250/250 [==============================] - 3s 10ms/step - loss: 0.6816 - accuracy: 0.5556 - val_loss: 0.6621 - val_accuracy: 0.5634
Epoch 2/10
250/250 [==============================] - 2s 8ms/step - loss: 0.5811 - accuracy: 0.6933 - val_loss: 0.5089 - val_accuracy: 0.8102
Epoch 3/10
250/250 [==============================] - 2s 8ms/step - loss: 0.3867 - accuracy: 0.8727 - val_loss: 0.3938 - val_accuracy: 0.8298
Epoch 4/10
250/250 [==============================] - 3s 10ms/step - loss: 0.2768 - accuracy: 0.9051 - val_loss: 0.3671 - val_accuracy: 0.8369
Epoch 5/10
250/250 [==============================] - 3s 10ms/step - loss: 0.2131 - accuracy: 0.9299 - val_loss: 0.3697 - val_accuracy: 0.8376
Epoch 6/10
250/250 [==============================] - 3s 10ms/step - loss: 0.1680 - accuracy: 0.9474 - val_loss: 0.3708 - val_accuracy: 0.8384
Epoch 7/10
250/250 [==============================] - 3s 10ms/step - loss: 0.1348 - accuracy: 0.9595 - val_loss: 0.3896 - val_accuracy: 0.8360
E

In [40]:
model.predict(test_sentences[0].reshape(1, -1))

1/1 [==============================] - 0s 65ms/step


array([[2.7085957e-06]], dtype=float32)

In [41]:
# Get the embedding layer from the model (i.e. first layer)
embedding_layer = model.layers[0]

# Get the weights of the embedding layer
embedding_weights = embedding_layer.get_weights()[0]

# Get the index-word dictionary
reverse_word_index = tokenizer.index_word

# Open writeable files
out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')

# Initialize the loop. Start counting at `1` because `0` is just for the padding
for word_num in range(1, 10000):

  # Get the word associated at the current index
  word_name = reverse_word_index[word_num]

  # Get the embedding weights associated with the current index
  word_embedding = embedding_weights[word_num]

  # Write the word name
  out_m.write(word_name + "\n")

  # Write the word embedding
  out_v.write('\t'.join([str(x) for x in word_embedding]) + "\n")

# Close the files
out_v.close()
out_m.close()

# Import files utilities in Colab
try:
  from google.colab import files
except ImportError:
  pass

# Download the files
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>